In [1]:
from pathlib import Path
from collections import Counter, defaultdict
from PIL import Image

import random, glob, logging
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from preprocess import is_img, parse_fake_path
from dataset import FakeImageDataset, RealImageDataset, collapse_for_classification

In [3]:
real_root = Path("/home/cpsc4710_yz2483/scratch_cpsc4710/cpsc4710_yz2483/animedl2m_dataset/real_images/resized_img")
fake_root = Path("/home/cpsc4710_yz2483/scratch_yz2483/animedl2m_dataset/fake_images")

In [ ]:
def collect_real_img(real_root: Path):
    try:
        ds = RealImageDataset(real_root)
        paths = getattr(ds, "paths", [])
    except Exception:
        raise ValueError(f"No matched files found in {real_root / "resized_img"}")

    real_map = {}
    for p in paths:
        subset = p.parent.stem
        id_ = p.stem
        if (subset, id_) in real_map:
            logging.warning(f"Duplicate real image for {(subset, id_)}: {p} vs {real_map[(subset, id_)]}")
        real_map[(subset, id_)] = p
        
    return real_map, paths

In [ ]:
fake_img_dict = parse_fake_path(fake_root, quiet=False)
collapsed_fake_img_dict = collapse_for_classification(fake_img_dict["records"])

In [ ]:
def _build_views(fake_root: Path, real_root: Path, collapsed_dict: dict):
    # construct dict that stores paths by (subset, id) -> {task -> {model -> path}}
    by_sid = {} # by (subset, id)
    for g in collapsed_dict:
        key_sid = (g["subset"], g["id"])
        by_sid.setdefault(key_sid, {"text2image": {}, "inpainting": {}})
        p = g.get("img_path")
        if isinstance(p, list) and p:
            p = p[0]
        if p and p.exists():
            by_sid[key_sid][g["task"]][g["model"]] = p

    real_map, real_paths = collect_real_img(real_root)
    return collapsed_dict, by_sid, real_map, len(real_paths)

In [5]:
fake_img_dict["records"][0]

{'subset': '0000',
 'id': '10000',
 'task': 'inpainting',
 'model': 'FLUX1',
 'inpaint_path': PosixPath('/home/cpsc4710_yz2483/scratch_yz2483/animedl2m_dataset/fake_images/0000/inpainting/10000/FLUX1_inpainting_10000_girl.png'),
 'mask_label': 'girl',
 'mask_path': PosixPath('/home/cpsc4710_yz2483/scratch_yz2483/animedl2m_dataset/fake_images/0000/mask/10000/girl.png'),
 'info': {'description': 'The image is a manga panel featuring three girls. The central girl is surprised, wearing a sailor-style school uniform. To her left, a girl in a dress looks down, and to her right, another girl in a school uniform holds a newspaper. The scene is monochromatic with a focus on their expressions and poses.',
  'objects': ['News', 'girl', 'character', 'accident', 'background']}}

In [ ]:
def make_sample_grid(fake_root: Path, real_root: Path, subset: str, id_: str, collapsed_fake_img_dict: dict,
                     savepath: Path | None = None, figsize=(10, 6)):
    MODELS = ["SD", "SDXL", "FLUX1"]
    _, by_sid, real_map, _ = _build_views(fake_root, real_root, collapsed_fake_img_dict)
    key_sid = (subset, id_)
    d = by_sid.get(key_sid, {"text2image": {}, "inpainting": {}})

    fig, axes = plt.subplots(2, 4, figsize=figsize)
    axes = axes.reshape(2, 4)

    def _show(ax, path, title):
        ax.axis("off")
        if path:
            from PIL import Image
            im = Image.open(path).convert("RGB")
            ax.imshow(im)
            ax.set_title(title, fontsize=10)
    
    _show(axes[0, 0], real_map.get(key_sid), f"Real Image\n{subset}/{id_}")
    for j, m in enumerate(MODELS, start=1):
        _show(axes[0, j], d["text2image"].get(m), f"Text2Image - {m}")
    
    for j, m in enumerate(MODELS):
        _show(axes[1, j], d["inpainting"].get(m), f"Inpainting - {m}")

    axes[1, 3].axis("off")
    
    plt.tight_layout()
    if savepath:
        plt.savefig(savepath, dpi=150, bbox_inches="tight")
    
    plt.show()

    